In [1]:
import math
from time import time
from sys import getsizeof
import os

### Making dictionary of basic factors

In [2]:
def sort_rename(sequence):
    """ Makes Name list and Pos dictionary for given sequence.
    
    :arg
        sequence: list of pairs or letters
        
    :returns
        - Name for `sequence` as list
        - Pos for `sequence` as dictionary
    """
    
    last_entry = None
    index = 0
    position_to_index = [None] * len(sequence)
    first_entry = {}
    for entry in sorted([(e, i) for i, e in enumerate(sequence)]):
        if last_entry and last_entry[0] != entry[0]:
            index += 1
            first_entry[index] = entry[1]

        position_to_index[entry[1]] = index
        if last_entry is None:
            first_entry[0] = entry[1]

        last_entry = entry

    return position_to_index, first_entry


def kmr(text):
    """ Karp-Miller-Rosenberg algorithm for making dictionary of basic factors
    
    :arg
        text: string for which DBF is being made
        
    :returns
        - dictionary of Name for every power of 2 less than length of `text`
        - dictionary of Pos for every power of 2 less than length of `text`
    """
    factor = math.floor(math.log2(len(text)))
    padding_length = 2 ** (factor + 1) - 1
    text += "}" * padding_length

    position_to_index, first_entry = sort_rename(list(text))
    names = {1: position_to_index}
    entries = {1: first_entry}

    for i in range(1, factor+1):
        power = 2 ** (i - 1)
        new_sequence = []
        for j in range(len(text)):
            if j+power < len(names[power]):
                new_sequence.append((names[power][j], names[power][j+power]))

        position_to_index, first_entry = sort_rename(new_sequence)
        names[power * 2] = position_to_index
        entries[power * 2] = first_entry

    return names, entries

### Finding patterns

In [3]:
def bin_search(text, pattern, entries, k, letter_index, start, end, start_or_end):
    """ Binary search returns first or last index of patterns starting with 
    `pattern[:(letter_index+1)]`
    
    :arg
        text:         text in which pattern is searched
        pattern:      pattern we are looking for
        entries:      list of Pos lists - part of DBF for `text`
        k:            power of 2 and length of part of `pattern` which is now considered
        letter_index: index of letter in `pattern` which is now considered
        start:        start index of `entries[k]`
        end:          end index of `entries[k]`
        start_or_end: whether binary search is looking for "start" or "end" index in 
                      `entries[k]` for `pattern[:(letter_index+1)]`
        
    :returns
        start or end index of interval of `entries[k]` which is contating indices of first
        occurrences of patterns starting with `pattern[:(letter_index+1)]`
    """
    curr_start = start
    curr_end = end

    middle = (curr_start + curr_end) // 2

    while curr_end - curr_start > 1:
        index = entries[k][middle]
        if (start_or_end == "start" and text[index + letter_index] >= pattern[letter_index] or
                start_or_end == "end" and text[index + letter_index] > pattern[letter_index]):
            curr_end = middle
        else:
            curr_start = middle

        middle = (curr_start + curr_end) // 2

    if start_or_end == "start":
        if (entries[k][curr_start] + letter_index < len(text) and
                text[entries[k][curr_start] + letter_index] == pattern[letter_index]):
            return curr_start
        return curr_end
    else:
        if (entries[k][curr_end] + letter_index < len(text) and
                text[entries[k][curr_end] + letter_index] == pattern[letter_index]):
            return curr_end
        return curr_start


def get_name(text, pattern, entries, k):
    """ Finds number which corresponds to first `k` letters of `pattern`.
    
    :arg
        text:    text in which the pattern will be searched
        pattern: all the pattern; just first `k` letters are taken into consideration
        entries: Pos lists in which there are indices of first occurrences of patterns
                 (pattersn are in alphabetical order)
        k:       power of 2 - number of first elements of `pattern` taken into consideration
    
    :returns
        `start` such that `entries[k][start]` is an index of first occurrence of the pattern 
        we are looking for 
    """
    entries_len = len(entries[k])
    curr_letter = 0

    start = 0
    end = entries_len - 1

    while start != end:
        start = bin_search(text, pattern, entries, k, curr_letter, start, end, "start")
        end = bin_search(text, pattern, entries, k, curr_letter, start, end, "end")
        curr_letter += 1

    return start


def get_text_from_file(file_path):
    """ Reads text from file
    
    :arg
        file_path: path of the file with text
        
    :returns
        contents of the file
    """
    with open(file_path, "r") as file:
        text = file.read()
        return text
    

def get_dbf(file_path):
    """ Gets dictionary of basic factors for text in file
    
    :arg
        file_path: path of the file with text
        
    :returns
        - Name as list of lists
        - Pos as list of lists
        - text read from file
    """
    text = get_text_from_file(file_path)
    names, entries = kmr(text)

    names = list(map(lambda item: item[1][:len(text)], names.items()))
    entries = list(map(lambda item: [item[1][e] for e in range(len(item[1])-1)], entries.items()))

    return names, entries, text


def find_pattern(text, pattern, names, entries):
    """ Finds all occurrences of pattern in text.
    
    :args
        text:           string - the place where pattern will be searched
        pattern:        pattern to match
        names, entries: DBF, results from `get_dbf` function
        
    :returns
        list of indices - starting points of all matchings
    """
    k = math.floor(math.log2(len(pattern)))
    
    # in case length of the pattern is not equal to power of 2, two patterns of length 2^k are
    # searched: one from the beginning of the pattern to index k-1 and second from index n-k
    # (where n is a length of the pattern) to the end of the pattern
    q = get_name(text, pattern, entries, k)

    part_size = 2 ** k
    if text[entries[k][q]:(entries[k][q]+part_size)] != pattern[:part_size]:
        return []

    # matchings with first part of the pattern
    patterns_indices = []
    for i in range(len(names[k])):
        if names[k][i] == q:
            patterns_indices.append(i)

    # second part of the pattern
    if len(pattern) != 2 ** k:
        q = get_name(text, pattern[(len(pattern) - part_size):], entries, k)

        if text[entries[k][q]:(entries[k][q] + part_size)] != pattern[(len(pattern) - part_size):]:
            return []

        right_patterns_indices = []

        for i in patterns_indices:
            if names[k][i + len(pattern) - part_size] == q:
                right_patterns_indices.append(i)

        return right_patterns_indices

    return patterns_indices

### McCreight algorithm

In [4]:
class Tree:
    """Structure containing suffix tree"""
    def __init__(self, word):
        self._root = Node(None, "")
        self._word = word

    def split_edge(self, parent_node, top_interval):
        # current edge
        curr_edge = parent_node.children[self.word[top_interval[0]]]

        # new node
        new_node = Node(parent_node, self.word[top_interval[0]])

        # edge from parent node to the new node
        top_edge = Edge(top_interval, new_node)
        parent_node.children[self.word[top_interval[0]]] = top_edge

        # edge from new node to the child of parent node
        bottom_interval = (top_interval[1] + 1, curr_edge.interval[1])
        bottom_edge = Edge(bottom_interval, curr_edge.node)
        new_node.add_child(self.word[top_interval[1] + 1], bottom_edge)
        curr_edge.node.parent = new_node
        curr_edge.node.parent_first_letter = self.word[top_interval[1] + 1]

    def find(self, word):
        curr_letter = 0
        curr_node = self.root
        while curr_letter < len(word):
            # checking if common part of word to find and text represented by the tree ended in the current node
            if word[curr_letter] not in curr_node.children:
                return False

            # looking for edge which is going out from current node
            curr_edge = curr_node.children[word[curr_letter]]
            interval = curr_edge.interval

            # checking current edge
            word_letter = interval[0]
            while word_letter <= interval[1]:
                if curr_letter == len(word):
                    return True

                if self.word[word_letter] != word[curr_letter]:
                    return False

                curr_letter += 1
                word_letter += 1

            curr_node = curr_edge.node

        return True

    @property
    def root(self):
        return self._root

    @property
    def word(self):
        return self._word


class Node:
    def __init__(self, parent, parent_first_letter):
        self._parent = parent
        self._parent_first_letter = parent_first_letter
        self._children = {}

        self._link = None

    def add_child(self, letter, edge):
        self._children[letter] = edge

    def print(self):
        print(self._children)
        for child in self._children.values():
            print(child.interval)
            child.node.print()

    @property
    def children(self):
        return self._children

    @property
    def parent(self):
        return self._parent

    @property
    def parent_first_letter(self):
        return self._parent_first_letter

    @property
    def link(self):
        return self._link

    @parent.setter
    def parent(self, parent):
        self._parent = parent

    @parent_first_letter.setter
    def parent_first_letter(self, parent_first_letter):
        self._parent_first_letter = parent_first_letter

    @link.setter
    def link(self, link):
        self._link = link


class Edge:
    def __init__(self, interval, node):
        self._interval = interval
        self._node = node

    @property
    def interval(self):
        return self._interval

    @property
    def node(self):
        return self._node


def split_edge(parent_node, top_interval, word):
    # current edge
    curr_edge = parent_node.children[word[top_interval[0]]]

    # new node
    new_node = Node(parent_node, word[top_interval[0]])

    # edge from parent node to the new node
    top_edge = Edge(top_interval, new_node)
    parent_node.children[word[top_interval[0]]] = top_edge

    # edge from new node to the child of parent node
    bottom_interval = (top_interval[1] + 1, curr_edge.interval[1])
    bottom_edge = Edge(bottom_interval, curr_edge.node)
    new_node.add_child(word[top_interval[1] + 1], bottom_edge)
    curr_edge.node.parent = new_node
    curr_edge.node.parent_first_letter = word[top_interval[1] + 1]


def graft(node, interval, first_letter):
    new_node = Node(node, first_letter)
    new_edge = Edge(interval, new_node)
    node.add_child(first_letter, new_edge)

    return new_node


def label_size(label):
    return label[1] - label[0] + 1


def fast_find(node, label, word):
    curr_node = node
    curr_label = label

    child_edge = curr_node.children[word[curr_label[0]]]
    while label_size(curr_label) > label_size(child_edge.interval):
        curr_node = child_edge.node
        curr_label = (curr_label[0] + label_size(child_edge.interval), curr_label[1])

        child_edge = curr_node.children[word[curr_label[0]]]

    if label_size(curr_label) == label_size(child_edge.interval):
        return child_edge.node
    else:
        new_node_top_label = (child_edge.interval[0], child_edge.interval[0] + label_size(curr_label) - 1)
        split_edge(curr_node, new_node_top_label, word)

        curr_node = curr_node.children[word[curr_label[0]]].node

        return curr_node


def slow_find(node, label, word):
    curr_node = node
    curr_label_letter = label[0]

    if not word[curr_label_letter] in curr_node.children:
        return curr_node, label

    child_edge = curr_node.children[word[curr_label_letter]]
    curr_edge_letter = child_edge.interval[0]

    while word[curr_label_letter] == word[curr_edge_letter]:
        if curr_edge_letter == child_edge.interval[1]:
            curr_node = child_edge.node
            curr_label_letter += 1
            if not word[curr_label_letter] in curr_node.children:
                return curr_node, (curr_label_letter, label[1])

            child_edge = curr_node.children[word[curr_label_letter]]
            curr_edge_letter = child_edge.interval[0]
        else:
            curr_label_letter += 1
            curr_edge_letter += 1

    new_node_top_interval = (child_edge.interval[0], curr_edge_letter - 1)
    split_edge(curr_node, new_node_top_interval, word)

    left_label = (curr_label_letter, label[1])
    return curr_node.children[word[child_edge.interval[0]]].node, left_label


def mccreight(word):
    tree = Tree(word)
    head = tree.root
    node = tree.root
    leaf = graft(node, (0, len(word)-1), word[0])

    for i in range(1, len(word)):
        left_label = (i, len(word)-1)
        if head == tree.root:
            node = tree.root
        else:
            to_head_label = head.parent.children[head.parent_first_letter].interval

            if head.parent == tree.root:
                to_head_label = (to_head_label[0] + 1, to_head_label[1])

                node = tree.root
            else:
                node = head.parent.link

            if to_head_label[1] >= to_head_label[0]:
                node = fast_find(node, to_head_label, word)
            left_label = (leaf.parent.children[leaf.parent_first_letter].interval[0], left_label[1])

        last_head = head
        head, left_label = slow_find(node, left_label, word)
        last_head.link = node
        leaf = graft(head, left_label, word[left_label[0]])

    return tree

### Knuth-Morris-Pratt algorithm

In [5]:
def prefix_function(pattern):
    pi = []
    pi.append(-1)
    k = -1
    for q in range(1, len(pattern)):
        while k >= 0 and pattern[k+1] != pattern[q]:
            k = pi[k]

        if pattern[k+1] == pattern[q]:
            k += 1

        pi.append(k)

    return pi


def kmp_matching(text, pattern, pi):
    pattern_shifts = []

    q = -1
    for i in range(len(text)):
        while q >= 0 and pattern[q+1] != text[i]:
            q = pi[q]

        if pattern[q+1] == text[i]:
            q += 1

        if q == len(pattern) - 1:
            pattern_shifts.append(i-q)
            q = pi[q]

    return pattern_shifts

### Tests - time of building structure

In [6]:
def print_time(function, to_print, get_result=False, *args):
    """ Prints time of `function` running.
    
    :arg
        function:   function which will be called
        to_print:   what should be printed before the time
        get_result: if the function should return result of `function`
        *args:      arguments of `function`
        
    :returns
        if `get_result` is True, then it returns result of `function`
    """
    start_time = time()
    result = function(*args)
    print(f"{to_print} {time() - start_time}s")

    if get_result:
        return result

#### Building structure for "1997_714.txt"

In [7]:
dbf_act = print_time(get_dbf, "dbf:         ", True, "1997_714.txt")
print_time(mccreight, "suffix tree: ", False, get_text_from_file("1997_714.txt") + "$")

dbf:          20.506614685058594s
suffix tree:  8.136092901229858s


#### Building structure for "romeo-i-julia-700.txt"

In [8]:
dbf_drama = print_time(get_dbf, "dbf:         ", True, "romeo-i-julia-700.txt")
print_time(mccreight, "suffix tree: ", False, get_text_from_file("romeo-i-julia-700.txt") + "$")

dbf:          0.4453434944152832s
suffix tree:  0.11213827133178711s


#### Building structure for "zad6"

In [9]:
dbf_zad = print_time(get_dbf, "dbf:         ", True, "zad6")
print_time(mccreight, "suffix tree: ", False, get_text_from_file("zad6") + "$")

dbf:          0.023640155792236328s
suffix tree:  0.007577657699584961s


### Tests - size of the structure

In [10]:
def get_size(structure):
    """ Returns size of the structure 
    
    :arg
        structure: size of that will be calculated; in case it is a type of list, tuple or 
                   string (with size grater than 1), then function will be called for every 
                   element in the collection
                   
    :returns
        size in bytes
    """
    size = getsizeof(structure)
    if type(structure) in (list, tuple, str) and not (type(structure) == str and len(structure) <= 1):
        for substructure in structure:
            size += get_size(substructure)
    
    return size

#### Size of  "1997_714.txt"

In [11]:
print(f"size of the file:      {os.stat('1997_714.txt').st_size} bytes")
print(f"size of the structure: {get_size(dbf_act)} bytes")

size of the file:      254133 bytes
size of the structure: 296008464 bytes


#### Size of "romeo-i-julia-700.txt"

In [12]:
print(f"size of the file:      {os.stat('romeo-i-julia-700.txt').st_size} bytes")
print(f"size of the structure: {get_size(dbf_drama)} bytes")

size of the file:      14208 bytes
size of the structure: 12378848 bytes


#### Size of "zad6"

In [13]:
print(f"size of the file:      {os.stat('zad6').st_size} bytes")
print(f"size of the structure: {get_size(dbf_zad)} bytes")

size of the file:      947 bytes
size of the structure: 645858 bytes


### Search in  "1997_714.txt"

In [14]:
pattern = "e"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_act[2], pattern, dbf_act[0], dbf_act[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_act[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.025287151336669922s
kmp:  0.05326581001281738s
Number of matchings found by dbf: 8433
Number of matchings found by kmp: 8433


In [15]:
pattern = "ustawy"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_act[2], pattern, dbf_act[0], dbf_act[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_act[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.02244734764099121s
kmp:  0.053707122802734375s
Number of matchings found by dbf: 32
Number of matchings found by kmp: 32


In [16]:
pattern = """Zrzeczenia, o którym mowa w ust. 1, podatnik dokonuje przez złożenie
  urzędowi skarbowemu właściwemu według miejsca zamieszkania podatnika
  pisemnego oświadczenia."""

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_act[2], pattern, dbf_act[0], dbf_act[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_act[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.02670764923095703s
kmp:  0.05548977851867676s
Number of matchings found by dbf: 1
Number of matchings found by kmp: 1


### Search in  "romeo-i-julia-700.txt"

In [17]:
pattern = "a"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_drama[2], pattern, dbf_drama[0], dbf_drama[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_drama[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.0011348724365234375s
kmp:  0.0027408599853515625s
Number of matchings found by dbf: 644
Number of matchings found by kmp: 644


In [18]:
pattern = "ROMEO"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_drama[2], pattern, dbf_drama[0], dbf_drama[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_drama[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.0013964176177978516s
kmp:  0.003291606903076172s
Number of matchings found by dbf: 16
Number of matchings found by kmp: 16


In [19]:
pattern = """Gdybyśmy mogli dojść tych trosk zarodka,
Nie zbrakłoby nam zaradczego środka."""

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_drama[2], pattern, dbf_drama[0], dbf_drama[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_drama[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.0016846656799316406s
kmp:  0.0030107498168945312s
Number of matchings found by dbf: 1
Number of matchings found by kmp: 1


### Search in  "zad6"

In [20]:
pattern = "p"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_zad[2], pattern, dbf_zad[0], dbf_zad[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_zad[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  9.846687316894531e-05s
kmp:  0.00020742416381835938s
Number of matchings found by dbf: 22
Number of matchings found by kmp: 22


In [21]:
pattern = "Zaimplementować"

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_zad[2], pattern, dbf_zad[0], dbf_zad[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_zad[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.0002655982971191406s
kmp:  0.0004124641418457031s
Number of matchings found by dbf: 2
Number of matchings found by kmp: 2


In [22]:
pattern = ("Porównać czas wyszukiwania wzorca przy użyciu DBF z wyszukiwaniem " + 
          "za pomocą KMP dla różnych długości wzorca.")

pattern_indices_dbf = print_time(find_pattern, "dbf: ", True, dbf_zad[2], pattern, dbf_zad[0], dbf_zad[1])
pattern_indices_kmp = print_time(kmp_matching, "kmp: ", True, dbf_zad[2], pattern, prefix_function(pattern))

print(f"Number of matchings found by dbf: {len(pattern_indices_dbf)}")
print(f"Number of matchings found by kmp: {len(pattern_indices_kmp)}")

dbf:  0.00014257431030273438s
kmp:  0.0002079010009765625s
Number of matchings found by dbf: 1
Number of matchings found by kmp: 1
